In [1]:
import pandas as pd, numpy as np

In [2]:
data = pd.read_csv('../data/FUF Salesforce Data Dump.csv')

/Users/spencer/Documents/Personal/datasci-street-tree-viz/venv/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61745 entries, 0 to 61744
Data columns (total 20 columns):
Tree Planting: Planting ID    61745 non-null object
Tree Species                  51502 non-null object
Tree Bldg Num                 59790 non-null object
Tree Street                   61371 non-null object
Tree Zip Code                 57992 non-null object
Neighborhood                  61694 non-null object
Cross Street                  41304 non-null object
Basin Type                    25983 non-null object
Permit #                      1763 non-null object
Survey2mDoneDate              4632 non-null object
Survey2mStatus                14435 non-null object
Care1Done                     27556 non-null object
2yrDone                       760 non-null object
Survey2yrStatus               747 non-null object
3yrDone                       21395 non-null object
Survey3yrStatus               18690 non-null object
4yrDone                       61 non-null object
Survey4yrStatus 

In [4]:
data.filter(regex='.*Status').count()

Survey2mStatus     14435
Survey2yrStatus      747
Survey3yrStatus    18690
Survey4yrStatus       60
Survey5yrStatus     3046
dtype: int64

In [5]:
data.filter(regex='.*Done').count()

Survey2mDoneDate     4632
Care1Done           27556
2yrDone               760
3yrDone             21395
4yrDone                61
5yrDone              3244
dtype: int64

- Status primarily available after 2m and 3yr, looks like it would be most useful to consider these two points in time

In [6]:
df_notna = data[['Tree Species', 'Neighborhood', 'Survey2mStatus', 'Survey3yrStatus']].dropna()

In [7]:
df_notna.describe()

,Tree Species,Neighborhood,Survey2mStatus,Survey3yrStatus
count,9964,9964,9964,9964
unique,243,45,9,6
top,Arbutus 'Marina',Outer Sunset,1 Very healthy,2 Good
freq,1017,855,6529,4103


In [8]:
df_notna['Survey2mStatus'].value_counts()

1 Very healthy          6529
2 Good                  2872
3 Struggling             497
4 Almost dead             56
5 Dead or gone             6
2                          1
1                          1
survey but no remark       1
-1                         1
Name: Survey2mStatus, dtype: int64

In [9]:
df_notna['Survey3yrStatus'].value_counts()

2 Good            4103
1 Very healthy    3972
3 Struggling      1224
5 Dead or gone     408
4 Almost dead      245
5 dead/gone         12
Name: Survey3yrStatus, dtype: int64

In [10]:
df_notna = df_notna[~df_notna['Survey2mStatus'].isin(['-1','survey but no remark'])]

In [11]:
di = {"1": 1, "1 Very healthy": 1, "2": 2, "2 Good": 2,
     "3": 3, '3 Struggling': 3, "4": 4, '4 Almost dead': 4,
     "5": 5, '5 Dead or gone': 5, '5 dead/gone': 5}

df_notna.replace(di, inplace = True)

df_notna['2m_3yr_change'] = df_notna['Survey3yrStatus'] - df_notna['Survey2mStatus']

In [12]:
df_notna['Survey2mStatus'].value_counts()

1    6530
2    2873
3     497
4      56
5       6
Name: Survey2mStatus, dtype: int64

In [13]:
df_notna['Survey3yrStatus'].value_counts()

2    4103
1    3970
3    1224
5     420
4     245
Name: Survey3yrStatus, dtype: int64

In [14]:
df_notna['2m_3yr_change'].value_counts()

 0    4148
 1    3053
-1    1270
 2     840
 3     267
 4     237
-2     136
-3      11
Name: 2m_3yr_change, dtype: int64

In [15]:
df_notna['2m_3yr_change'].describe()

count    9962.000000
mean        0.492572
std         1.102331
min        -3.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         4.000000
Name: 2m_3yr_change, dtype: float64

In [16]:
df_notna[df_notna['2m_3yr_change']==-4]['Neighborhood'].value_counts().head()

Series([], Name: Neighborhood, dtype: int64)

In [41]:
df_notna[df_notna['2m_3yr_change']==-4]['Tree Species'].value_counts().head()

Series([], Name: Tree Species, dtype: int64)

In [42]:
df_notna['2m_3yr_change_category'] = df_notna['2m_3yr_change'].apply(lambda x: 'Better' if x in [1,2,3] else
                                                                                'Worse or Same')
df_notna['2m_3yr_change_binary'] = df_notna['2m_3yr_change'].apply(lambda x: '1' if x in [1,2,3] else
                                                                                '0')

In [43]:
df_notna['2m_3yr_change_category'].value_counts()

Worse or Same    5802
Better           4160
Name: 2m_3yr_change_category, dtype: int64

In [44]:
df_notna.groupby('2m_3yr_change_category').describe()

2m_3yr_change  Survey2mStatus  Survey3yrStatus
2m_3yr_change_category                                                      
Better                 count    4160.000000     4160.000000      4160.000000
                       mean        1.330288        1.181010         2.511298
                       std         0.591310        0.440414         0.791496
                       min         1.000000        1.000000         2.000000
                       25%         1.000000        1.000000         2.000000
                       50%         1.000000        1.000000         2.000000
                       75%         2.000000        1.000000         3.000000
                       max         3.000000        4.000000         5.000000
Worse or Same          count    5802.000000     5802.000000      5802.000000
                       mean       -0.108066        1.569804         1.461737
                       std         0.985784        0.673869         0.888549
                       min        -3.000000        1.000000         1.000000
                       25%         0.000000        1.000000         1.000000
                       50%         0.000000        1.000000         1.000000
                       75%         0.000000        2.000000         2.000000
                       max         4.000000        5.000000         5.000000

In [45]:
df_notna.ix[df_notna['Tree Species']<>'', 
            'genus'] = df_notna[df_notna['Tree Species']<>'']['Tree Species'].apply(lambda x: x.split()[0])

In [46]:
df_notna[df_notna['2m_3yr_change']==-4]['genus'].value_counts().head()

Series([], Name: genus, dtype: int64)

In [47]:
df_notna.head()

,Tree Species,Neighborhood,Survey2mStatus,Survey3yrStatus,2m_3yr_change,2m_3yr_change_category,2m_3yr_change_binary,genus
25,Prunus salicina 'Santa Rosa',Bayview,1,2,1,Better,1,Prunus
49,Prunus yedoensis 'Akebono',Mission,1,1,0,Worse or Same,0,Prunus
52,Prunus yedoensis 'Akebono',Mission,1,1,0,Worse or Same,0,Prunus
58,Rhamnus alaternus,Outer Sunset,1,1,0,Worse or Same,0,Rhamnus
61,Rhamnus alaternus,Outer Sunset,1,1,0,Worse or Same,0,Rhamnus


In [48]:
df_notna.groupby('genus')['2m_3yr_change'].mean().reset_index().sort('2m_3yr_change').head()

/Users/spencer/Documents/Personal/datasci-street-tree-viz/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,genus,2m_3yr_change
80,Robinia,-1.000000
30,Dracaena,-0.333333
89,Washingtonia,-0.222222
45,Ilex,0.000000
13,Carpinus,0.000000


In [49]:
df_notna.groupby('Neighborhood')['2m_3yr_change'].mean().reset_index().sort('2m_3yr_change').head()

/Users/spencer/Documents/Personal/datasci-street-tree-viz/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Neighborhood,2m_3yr_change
20,Mission Terrace,-0.235294
41,Upper Market,-0.178571
39,Twin Peaks,-0.111111
11,Glen Park,-0.093750
8,Duboce Triangle/Lower Haight,0.000000


In [50]:
df_notna.head()

,Tree Species,Neighborhood,Survey2mStatus,Survey3yrStatus,2m_3yr_change,2m_3yr_change_category,2m_3yr_change_binary,genus
25,Prunus salicina 'Santa Rosa',Bayview,1,2,1,Better,1,Prunus
49,Prunus yedoensis 'Akebono',Mission,1,1,0,Worse or Same,0,Prunus
52,Prunus yedoensis 'Akebono',Mission,1,1,0,Worse or Same,0,Prunus
58,Rhamnus alaternus,Outer Sunset,1,1,0,Worse or Same,0,Rhamnus
61,Rhamnus alaternus,Outer Sunset,1,1,0,Worse or Same,0,Rhamnus


In [51]:
df_notna.genus.nunique()

90

In [52]:
df_notna.Neighborhood.nunique()

45

In [53]:
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline

In [54]:
logistic = linear_model.LogisticRegression()

In [55]:
X_full = pd.get_dummies(df_notna[['genus', 'Neighborhood']], prefix=['genus', 'Neighborhood'])
y_full = df_notna['2m_3yr_change_binary']

n_samples = len(X_full)

X_train = X_full[:int(.6 * n_samples)]
y_train = y_full[:int(.6 * n_samples)]
X_test = X_full[int(.6 * n_samples):]
y_test = y_full[int(.6 * n_samples):]


model = logistic.fit(X_train, y_train)
score = model.score(X_test, y_test)
print(score)

0.55382685069


In [69]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.57      0.81      0.67      2225
          1       0.49      0.23      0.31      1760

avg / total       0.53      0.55      0.51      3985



In [71]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1805,  420],
       [1358,  402]])

In [56]:
print(model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [57]:
X_train.columns

Index([u'genus_Acacia', u'genus_Acer', u'genus_Aesculus', u'genus_Agonis',
       u'genus_Albizia', u'genus_Alnus', u'genus_Aloe', u'genus_Arbutus',
       u'genus_Arctostaphylos', u'genus_Betula',
       ...
       u'Neighborhood_South Central Sunset',
       u'Neighborhood_Sunnyside/Balboa Terrace',
       u'Neighborhood_Telegraph Hill', u'Neighborhood_Tenderloin',
       u'Neighborhood_Twin Peaks', u'Neighborhood_USF/NoPa',
       u'Neighborhood_Upper Market', u'Neighborhood_Visitacion Valley',
       u'Neighborhood_West Portal',
       u'Neighborhood_Western Addition/Hayes Valley'],
      dtype='object', length=135)

In [58]:
model.coef_

array([[-0.14944186,  0.10303944, -0.07710943,  0.22173242,  0.20757989,
        -0.72285664,  0.        ,  0.08860858,  0.57108571, -1.2141253 ,
         0.        , -0.6999782 ,  0.3523533 , -0.32486804,  0.54457783,
         0.55914703, -0.1334146 ,  0.03855524, -0.13238628,  0.05610557,
        -0.29169415, -0.26907456,  0.49857011,  0.91141756, -0.7017463 ,
        -0.03719846,  0.74016652,  0.        ,  0.        ,  0.48738896,
        -0.23608499, -0.43279771, -0.20299121,  0.04287918,  1.02569555,
         0.36486187,  0.06639343, -0.86410569,  0.16579524,  0.34512673,
         1.2206883 , -0.50159805, -0.38620096,  0.85330596, -0.51666416,
        -0.35142313, -0.02785376,  0.01363124,  0.21742526, -0.69053843,
         0.93021684,  0.13223272,  0.66966548, -0.04481463, -0.24744492,
        -0.25168859, -0.34177128,  0.19338251,  0.73369915,  0.48085371,
         0.1699915 , -0.36060724,  0.15991044, -0.41254736, -0.32407083,
         0.41827256, -0.24744492,  0.        , -0.4

In [59]:
results = [model.coef_,X_train.columns]
results

[array([[-0.14944186,  0.10303944, -0.07710943,  0.22173242,  0.20757989,
         -0.72285664,  0.        ,  0.08860858,  0.57108571, -1.2141253 ,
          0.        , -0.6999782 ,  0.3523533 , -0.32486804,  0.54457783,
          0.55914703, -0.1334146 ,  0.03855524, -0.13238628,  0.05610557,
         -0.29169415, -0.26907456,  0.49857011,  0.91141756, -0.7017463 ,
         -0.03719846,  0.74016652,  0.        ,  0.        ,  0.48738896,
         -0.23608499, -0.43279771, -0.20299121,  0.04287918,  1.02569555,
          0.36486187,  0.06639343, -0.86410569,  0.16579524,  0.34512673,
          1.2206883 , -0.50159805, -0.38620096,  0.85330596, -0.51666416,
         -0.35142313, -0.02785376,  0.01363124,  0.21742526, -0.69053843,
          0.93021684,  0.13223272,  0.66966548, -0.04481463, -0.24744492,
         -0.25168859, -0.34177128,  0.19338251,  0.73369915,  0.48085371,
          0.1699915 , -0.36060724,  0.15991044, -0.41254736, -0.32407083,
          0.41827256, -0.24744492,  0.

In [60]:
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing, metrics, cross_validation
from sklearn.model_selection import cross_val_score


In [61]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_full, y_full, cv=5)
scores

array([ 0.58655294,  0.58855996,  0.60441767,  0.57128514,  0.54267068])

In [62]:
from sklearn import metrics

model.fit(X_train, y_train)
pred = model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label = 0)
roc_auc = auc(fpr, tpr)
roc_auc

nan

In [63]:
y_test

35400    1
35489    0
35491    1
35493    0
35497    0
35499    1
35503    0
35505    1
35507    1
35545    0
35547    0
35551    1
35555    1
35559    0
35561    1
35569    0
35571    0
35573    1
35577    1
35581    0
35585    0
35589    1
35991    0
35993    1
35995    1
35997    0
36003    0
36005    0
36007    1
36025    1
        ..
60399    1
60400    0
60405    1
60412    0
60414    0
60415    0
60416    1
60417    0
60418    0
60419    0
60426    0
60439    1
60440    0
60442    1
60443    1
60452    0
60454    0
60456    0
60457    0
60458    0
60459    1
60461    0
60466    0
60467    1
60473    1
60474    0
60494    1
60502    1
60509    0
61300    1
Name: 2m_3yr_change_binary, dtype: object

In [64]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
labels = np.array([0, 0, 2, 2])
label_kfold = LabelKFold(labels, n_folds=2)
len(label_kfold)
2
print(label_kfold)

NameError: name 'LabelKFold' is not defined

In [ ]:

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
labels = np.array([0, 0, 2, 2])
label_kfold = LabelKFold(labels, n_folds=2)
len(label_kfold)
2
print(label_kfold)
sklearn.cross_validation.LabelKFold(n_labels=4, n_folds=2)
for train_index, test_index in label_kfold:
print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X_train, X_test, y_train, y_test)

In [ ]:
# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=6)
logistic = linear_model.LogisticRegression()

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)

colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange'])
lw = 2

i = 0
for (train, test), color in zip(cv.split(X_full, y_full), colors):
    probas_ = logistic.fit(X_full[train], y_full[train]).predict_proba(X_full[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_full[test], probas_[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=color,
             label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',
         label='Luck')

mean_tpr /= cv.get_n_splits(X_full, y_full)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='g', linestyle='--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()